In [6]:
import math
import logging
import numpy as np
import pandas as pd
import scipy.signal as sig
import altair as alt

from src.spinorama.load import graph_melt
from src.spinorama.load.parse import parse_graphs_speaker, normalize
from src.spinorama.load.rewseq import parse_eq_iir_rews
from src.spinorama.graph import graph_spinorama, graph_freq
from src.spinorama.normalize import normalize_mean, normalize_cea2034
from src.spinorama.scores import speaker_pref_rating
from src.spinorama.cea2034 import compute_cea2034, estimated_inroom_HV, spl2pressure, pressure2spl
from src.spinorama.iir import Biquad

In [7]:
# df_speaker = parse_graphs_speaker('Genelec', 'Genelec 8341A')
# df_speaker = parse_graphs_speaker('Focal', 'Focal Aria 906')
# df_speaker = parse_graphs_speaker('Elac', 'Elac Debut 2.0 B6.2')
df_speaker = parse_graphs_speaker('Adam', 'Adam S2V')

g_params = {'xmin': 20, 'xmax': 20000, 'ymin': -50, 'ymax': 10, 'width': 400, 'height': 250}
# graph_freq(df_speaker['SPL Horizontal'], g_params) | graph_freq(df_speaker['SPL Vertical'], g_params)
spin  = df_speaker['CEA2034_unmelted']
pir   = estimated_inroom_HV(df_speaker['SPL Horizontal_unmelted'], df_speaker['SPL Vertical_unmelted'])
score = speaker_pref_rating(graph_melt(spin), graph_melt(pir))

In [8]:
def parametric_eq_build(freq, peq):
    filter = 0
    for w, iir in peq:
        filter += w*np.array([iir.log_result(f) for f in freq])
    return filter


def parametric_eq_apply(spl, peq):
    filter = 0
    for w, iir in peq:
        filter += w*np.array([iir(v) for v in spl])
    return filter


def iir_graph(spin, measurement, peq):
    spin_freq   = spin['Freq'].to_numpy()
    mean = np.mean(spin.loc[(spin.Freq>500) & (spin.Freq<10000)]['On Axis'])
    curve = spin[measurement]-mean
    filter = parametric_eq_build(spin_freq, peq)
    curve_filtered = parametric_eq_apply(curve, peq)
    dff = pd.DataFrame({'Freq': spin_freq, measurement: curve, '{0} Filtered'.format(measurement): curve_filtered, 'Filter': filter})
    return alt.Chart(graph_melt(dff)).mark_line(clip=True).encode(
        x=alt.X('Freq:Q', scale=alt.Scale(type='log', nice=False, domain=[20, 20000])),
        y=alt.Y('dB:Q', scale=alt.Scale(domain=[-25, 5])),
        color=alt.Color('Measurements')
    )

def iir_apply(spl, peq):
    freq   = spl['Freq'].to_numpy()
    mean = np.mean(spl.loc[(spin.Freq>500) & (spl.Freq<10000)]['On Axis'])
    ddf = []
    ddf.append(pd.DataFrame({'Freq': freq}))
    for angle in spl.keys():
        if angle == 'Freq':
            continue
        curve = spl[angle]-mean
        # curve_filtered = curve+parametric_eq_apply(curve, peq)
        curve_filtered = curve+parametric_eq_build(freq, peq)
        # print('{0:7s} range [{1:.1f}, {2:.1f}] filtered [{3:.1f}, {4:.1f}]'.format(angle, np.min(curve), np.max(curve), np.min(curve_filtered), np.max(curve_filtered)))
        # print(curve, curve_filtered)
        ddf.append(pd.DataFrame({angle: curve_filtered}))
    return pd.concat(ddf, axis=1)



In [9]:
my_fs = 48000
my_peq = [
    (0, Biquad(Biquad.HIGHPASS, 50, my_fs, 0.2, 1)),
    (1, Biquad(Biquad.PEAK, 715, my_fs, 8, -2.8)),
    (0, Biquad(Biquad.NOTCH, 1400, my_fs, 0.1, 0.1)),
]

my_peq = parse_eq_iir_rews('./datas/eq/Adam S2V/iir.txt', my_fs)
ddf_horizontal = iir_apply(df_speaker['SPL Horizontal_unmelted'], my_peq)
ddf_vertical   = iir_apply(df_speaker['SPL Vertical_unmelted'], my_peq)

spin_filtered = normalize(graph_melt(compute_cea2034(ddf_horizontal, ddf_vertical)))
pir_filtered  = normalize(graph_melt(estimated_inroom_HV(ddf_horizontal, ddf_vertical)))
score_filtered = speaker_pref_rating(spin_filtered, pir_filtered)

In [10]:
for i in range(0, len(my_peq)):
    if my_peq[i][0] != 0:
        print(my_peq[i][1])
print('         SPK FLT')
print('-----------------')
print('NBD  ON {0:0.2f} {1:0.2f}'.format(score['nbd_on_axis'], score_filtered['nbd_on_axis']))
print('NBD  LW {0:0.2f} {1:0.2f}'.format(score['nbd_listening_window'], score_filtered['nbd_listening_window']))
print('NBD PIR {0:0.2f} {1:0.2f}'.format(score['nbd_pred_in_room'], score_filtered['nbd_pred_in_room']))
print('SM  PIR {0:0.2f} {1:0.2f}'.format(score['sm_pred_in_room'], score_filtered['sm_pred_in_room']))
print('SM   SP {0:0.2f} {1:0.2f}'.format(score['sm_sound_power'], score_filtered['sm_sound_power']))
print('LFX       {0}   {1}'.format(score['lfx_hz'], score_filtered['lfx_hz']))
print('LFQ     {0} {1}'.format(score['lfq'], score_filtered['lfq']))
print('-----------------')
print('Score    {0}  {1}'.format(score['pref_score'], score_filtered['pref_score']))
print('-----------------')

Type:3,Freq:340.0,Rate:48000.0,Q:10.0,Gain:0.8
Type:3,Freq:485.0,Rate:48000.0,Q:10.0,Gain:1.5
Type:3,Freq:597.0,Rate:48000.0,Q:5.0,Gain:-1.5
Type:3,Freq:1049.0,Rate:48000.0,Q:1.0,Gain:-1.8
Type:3,Freq:2902.0,Rate:48000.0,Q:4.2,Gain:-1.1
Type:3,Freq:3979.0,Rate:48000.0,Q:5.0,Gain:1.4
Type:3,Freq:5886.0,Rate:48000.0,Q:5.0,Gain:3.1
Type:3,Freq:6702.0,Rate:48000.0,Q:1.0,Gain:-3.8
Type:3,Freq:7943.0,Rate:48000.0,Q:5.0,Gain:3.1
Type:3,Freq:8825.0,Rate:48000.0,Q:5.0,Gain:-2.7
Type:3,Freq:9458.0,Rate:48000.0,Q:5.0,Gain:2.8
Type:3,Freq:10680.0,Rate:48000.0,Q:10.0,Gain:-0.7
Type:3,Freq:11866.0,Rate:48000.0,Q:1.0,Gain:-1.8
Type:3,Freq:17828.0,Rate:48000.0,Q:5.0,Gain:-1.6
         SPK FLT
-----------------
NBD  ON 0.34 0.24
NBD  LW 0.32 0.19
NBD PIR 0.29 0.20
SM  PIR 0.71 0.93
SM   SP 0.92 0.95
LFX       43   43
LFQ     0.77 0.74
-----------------
Score    5.5  6.6
-----------------


In [ ]:
graph_spinorama(df_speaker['CEA2034'], g_params) | graph_spinorama(spin_filtered, g_params)

In [ ]:
my_fs = 96000
my_peq = [
    (0, Biquad(Biquad.HIGHPASS, 20, my_fs, 0.2, 1)),
    (0, Biquad(Biquad.PEAK, 760, my_fs, 8, -2)),
    (1, Biquad(Biquad.BANDPASS, 1400, my_fs, 0.1, 3)),
]

freq = spin.Freq
db = [1] * len(freq)
# db = [math.sin(i) for i in range(0, len(db))]
df_flat = pd.DataFrame({'Freq': freq, 'On Axis': db})
df_filt = iir_apply(df_flat, my_peq)
#print(df_filt)
print(df_filt['Freq'].min(), df_filt['Freq'].max())
print(df_filt['On Axis'].min(), df_filt['On Axis'].max())
iir_graph(df_filt, 'On Axis', my_peq) | graph_freq(graph_melt(df_filt), g_params)

In [ ]:
def score_with_filter(splH, splV, peq):
    ddf_horizontal = iir_apply(splH, peq)
    ddf_vertical   = iir_apply(splV, peq)
    spin_filtered = normalize(graph_melt(compute_cea2034(ddf_horizontal, ddf_vertical)))
    pir_filtered  = normalize(graph_melt(estimated_inroom_HV(ddf_horizontal, ddf_vertical)))
    score_filtered = speaker_pref_rating(spin_filtered, pir_filtered, rounded=False)
    return -score_filtered['pref_score']

In [ ]:
my_fs = 48000
my_peq = [
    (0, Biquad(Biquad.HIGHPASS, 50, my_fs, 0.2, 1)),
    (1, Biquad(Biquad.PEAK, 715, my_fs, 8, -2.8)),
    (0, Biquad(Biquad.NOTCH, 1400, my_fs, 0.1, 0.1)),
]
score_with_filter(df_speaker['SPL Horizontal_unmelted'], df_speaker['SPL Vertical_unmelted'], my_peq)

In [ ]:
x0 = [700, 6, -2]

def objective(params):
    freq = params[0]
    Q    = params[1]
    Z    = params[2]
    the_peq = [(1, Biquad(Biquad.PEAK, freq, my_fs, Q, Z))]
    score = score_with_filter(df_speaker['SPL Horizontal_unmelted'], df_speaker['SPL Vertical_unmelted'], the_peq)
    print(params, score)
    return score

In [ ]:
objective(x0)

In [ ]:
from scipy.optimize import minimize, basinhopping

In [ ]:
bounds = [(20, 20000), (-10, 10), (0.1, 2)]
res = minimize(objective, x0, method='SLSQP', bounds=bounds)

res.x


In [ ]:

minimizer_kwargs = {"method": "COBYLA"}

def objective(params):
    freq = params[0]
    Q    = params[1]
    Z    = params[2]
    the_peq = [(1, Biquad(Biquad.PEAK, freq, my_fs, Q, Z))]
    score = score_with_filter(df_speaker['SPL Horizontal_unmelted'], df_speaker['SPL Vertical_unmelted'], the_peq)
    print(params, score)
    return score

class MyTakeStep(object):
   def __init__(self, stepsize=0.1):
       self.stepsize = stepsize
   def __call__(self, x):
       s = self.stepsize
       x[0] += np.random.uniform(-1000.*s, 1000.*s)
       x[1:] += np.random.uniform(-s, s, x[1:].shape)
       return x

class MyBounds(object):
    def __init__(self, xmax=[20000,10, 10], xmin=[20,-10, -10] ):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin

mytakestep = MyTakeStep()
mybounds = MyBounds()

x0 = [700, 5, -2]
res = basinhopping(objective2, 
                   x0, 
                   minimizer_kwargs=minimizer_kwargs, 
                   take_step=mytakestep, 
                   accept_test=mybounds,
                   interval=5,
                   niter=2)